In [ ]:
# 📦 Cell 1: Install Dependencies and Imports

# Install PyTorch if needed
try:
    import torch
    print("✅ PyTorch already installed")
except ImportError:
    print("📦 Installing PyTorch...")
    %pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
    import torch

# All necessary imports
import torch.nn as nn
import torch.optim as optim
import logging
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import joblib
import json
import time
import os
import sqlite3
from typing import Dict, List, Tuple
from datetime import datetime, timedelta
import difflib
from collections import defaultdict

# Set up device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🔥 Using device: {device}")
if torch.cuda.is_available():
    print(f"🚀 GPU: {torch.cuda.get_device_name(0)}")

print("✅ All imports loaded successfully!")


In [ ]:
# 🧠 Cell 2: PyTorch Classes and Utilities

class KillPredictionDataset(Dataset):
    """PyTorch dataset for kill prediction"""
    def __init__(self, X: np.ndarray, y: np.ndarray):
        self.X = torch.FloatTensor(X)
        self.y = torch.FloatTensor(y)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

class KillPredictionNN(nn.Module):
    """Neural network for kill prediction"""
    def __init__(self, input_size: int, hidden_sizes: List[int] = [128, 64, 32]):
        super(KillPredictionNN, self).__init__()

        layers = []
        prev_size = input_size

        for hidden_size in hidden_sizes:
            layers.extend([
                nn.Linear(prev_size, hidden_size),
                nn.ReLU(),
                nn.Dropout(0.2),
                nn.BatchNorm1d(hidden_size)
            ])
            prev_size = hidden_size

        layers.append(nn.Linear(prev_size, 1))
        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)

def check_database_schema(db_path):
    """Check if database has required tables"""
    if not os.path.exists(db_path):
        print(f"❌ Database file not found: {db_path}")
        return False
    try:
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
        tables = [row[0] for row in cursor.fetchall()]
        required_tables = ['players', 'matches', 'teams', 'player_match_stats']
        missing_tables = [table for table in required_tables if table not in tables]
        if missing_tables:
            print(f"⚠️ Missing tables: {missing_tables}")
        else:
            print(f"✅ Found all required tables: {required_tables}")
        conn.close()
        return True
    except Exception as e:
        print(f"❌ Database error: {e}")
        return False

print("✅ PyTorch classes loaded!")


In [ ]:
# 📁 Cell 3: Upload Database File

from google.colab import files
print("📤 Please upload your valorant_matches.db file:")
uploaded = files.upload()

if uploaded:
    db_path = list(uploaded.keys())[0]
    print(f"✅ Database uploaded: {db_path}")
    
    # Quick verification
    file_size = os.path.getsize(db_path) / (1024 * 1024)  # MB
    print(f"📊 File size: {file_size:.2f} MB")
    
    if check_database_schema(db_path):
        print("✅ Database structure verified!")
    else:
        print("⚠️ Database structure check failed, but continuing...")
else:
    print("❌ No file uploaded")


In [ ]:
# ⚡ Cell 4: Ultra-Fast Classes (Required for Advanced Training)

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class FastDatabaseDataLoader:
    def __init__(self, db_path: str):
        self.db_path = db_path
        self.scaler = StandardScaler()
        self.feature_columns = None

    def get_connection(self):
        return sqlite3.connect(self.db_path)

    def calculate_map_features_FAST(self, df: pd.DataFrame) -> pd.DataFrame:
        """⚡ ULTRA-FAST vectorized feature engineering"""
        print("⚡ Starting ULTRA-FAST feature engineering...")
        start_time = time.time()
        
        # Convert and sort
        df['match_date'] = pd.to_datetime(df['match_date'])
        df = df.sort_values(['consolidated_player_name', 'match_date', 'map_id']).reset_index(drop=True)

        # ⚡ VECTORIZED FEATURES (NO LOOPS!)
        print("⚡ Calculating vectorized features...")
        
        # Historical averages using rolling with shift (NO DATA LEAKAGE!)
        df['hist_avg_kills'] = (
            df.groupby('consolidated_player_name')['kills']
            .rolling(10, min_periods=1).mean()
            .shift(1).reset_index(level=0, drop=True)
        ).fillna(15.0)
        
        df['hist_avg_kdr'] = (
            df.groupby('consolidated_player_name')['kdr']
            .rolling(10, min_periods=1).mean()
            .shift(1).reset_index(level=0, drop=True)
        ).fillna(1.0)

        # Recent form
        df['recent_kills_5'] = (
            df.groupby('consolidated_player_name')['kills']
            .rolling(5, min_periods=1).mean()
            .shift(1).reset_index(level=0, drop=True)
        ).fillna(df['hist_avg_kills'])

        # Other features
        df['days_since_last'] = df.groupby('consolidated_player_name')['match_date'].diff().dt.days.fillna(7.0)

        # Series importance
        series_importance = {'bo1': 1, 'bo3': 2, 'bo5': 3}
        df['series_importance'] = df['series_type'].map(series_importance).fillna(1)
        
        df = df.fillna(0)
        
        elapsed = time.time() - start_time
        print(f"🎉 ULTRA-FAST feature engineering completed in {elapsed:.1f} seconds!")
        return df

class PlayerConsolidator:
    """Consolidates duplicate players and manages player identity"""
    
    def __init__(self, similarity_threshold=0.85):
        self.similarity_threshold = similarity_threshold
        self.player_mapping = {}  # original_name -> consolidated_name
        self.consolidated_players = {}  # consolidated_name -> list of original names
        
    def normalize_name(self, name):
        """Normalize player name for comparison"""
        if pd.isna(name) or name is None:
            return "unknown_player"
        
        # Convert to lowercase and remove common suffixes/prefixes
        normalized = str(name).lower().strip()
        
        # Remove team tags and common suffixes
        removals = ['_sen', '_c9', '_100t', '_nv', '_tsm', '_lg', '_faze', 
                   ' (sentinels)', ' (cloud9)', ' (100 thieves)', ' (envy)',
                   ' (team solomid)', ' (luminosity)', ' (faze)', '_v1',
                   ' (version1)', ' (g2)', '_g2', ' (acend)', '_ace']
        
        for removal in removals:
            normalized = normalized.replace(removal, '')
        
        # Remove special characters but keep spaces
        import re
        normalized = re.sub(r'[^\w\s]', '', normalized)
        normalized = re.sub(r'\s+', ' ', normalized).strip()
        
        return normalized
    
    def find_similar_players(self, player_names):
        """Find groups of similar player names that should be consolidated"""
        print(f"🔍 Analyzing {len(set(player_names))} unique player names for duplicates...")
        
        unique_names = list(set(player_names))
        normalized_names = {name: self.normalize_name(name) for name in unique_names}
        
        # Group by exact normalized match first
        exact_groups = defaultdict(list)
        for original, normalized in normalized_names.items():
            exact_groups[normalized].append(original)
        
        # Find fuzzy matches for remaining singles
        consolidated_groups = []
        processed = set()
        
        for normalized, originals in exact_groups.items():
            if len(originals) > 1:
                # Multiple names normalize to same thing - definitely duplicates
                consolidated_groups.append(originals)
                processed.update(originals)
        
        return consolidated_groups
    
    def consolidate_players(self, df):
        """Consolidate duplicate players in the dataframe"""
        print("\\n🔄 CONSOLIDATING DUPLICATE PLAYERS...")
        
        original_players = df['player_name'].nunique()
        
        # Find duplicate groups
        duplicate_groups = self.find_similar_players(df['player_name'].unique())
        
        # Create mapping from original to consolidated name
        for group in duplicate_groups:
            # Use the most common name as the consolidated name
            name_counts = df[df['player_name'].isin(group)]['player_name'].value_counts()
            consolidated_name = name_counts.index[0]  # Most frequent name
            
            for original_name in group:
                self.player_mapping[original_name] = consolidated_name
            
            self.consolidated_players[consolidated_name] = group
        
        # Apply consolidation
        df['consolidated_player_name'] = df['player_name'].map(
            lambda x: self.player_mapping.get(x, x)
        )
        
        # Report consolidation results
        final_players = df['consolidated_player_name'].nunique()
        duplicates_merged = original_players - final_players
        
        print(f"✅ CONSOLIDATION COMPLETE:")
        print(f"   Original players: {original_players}")
        print(f"   Final players: {final_players}")
        print(f"   Duplicates merged: {duplicates_merged}")
        
        return df

print("✅ Ultra-fast classes loaded!")


In [ ]:
# 🎯 Cell 5: ADVANCED FEATURE TRAINING (THE MAIN ONE!)

print("🎯 ENHANCED FEATURE ENGINEERING FOR SUB-3 MAE")
print("Now with Head-to-Head matchups, Recent Form Momentum, and Map-Opponent Analysis!")
print("")

class AdvancedFeatureEngineer:
    """Advanced feature engineering for esports prediction"""
    
    def calculate_matchup_features(self, df):
        """Calculate matchup evenness and game flow features"""
        print("🔍 CALCULATING MATCHUP DYNAMICS...")
        
        # 1. ROUND COUNT ANALYSIS (More rounds = more kills)
        match_round_stats = df.groupby('match_id').agg({
            'kills': ['sum', 'count'],
            'series_type': 'first'
        }).reset_index()
        
        match_round_stats.columns = ['match_id', 'total_kills', 'total_records', 'series_type']
        match_round_stats['estimated_rounds_per_map'] = match_round_stats['total_kills'] / match_round_stats['total_records']
        
        # Categorize game length
        def categorize_game_length(rounds_per_map):
            if rounds_per_map >= 16:
                return 'long_game'
            elif rounds_per_map >= 12:
                return 'medium_game'
            else:
                return 'short_game'
        
        match_round_stats['game_length_category'] = match_round_stats['estimated_rounds_per_map'].apply(categorize_game_length)
        
        # 2. MATCHUP EVENNESS (Close games vs stomps)
        match_variance = df.groupby('match_id')['kills'].agg(['std', 'mean']).reset_index()
        match_variance['kill_cv'] = match_variance['std'] / match_variance['mean']
        
        def categorize_evenness(cv):
            if pd.isna(cv):
                return 'unknown'
            elif cv <= 0.4:
                return 'even_matchup'
            elif cv <= 0.7:
                return 'somewhat_even'
            else:
                return 'uneven_stomp'
        
        match_variance['matchup_evenness'] = match_variance['kill_cv'].apply(categorize_evenness)
        
        # 3. SERIES PRESSURE
        series_pressure = {'bo1': 3, 'bo3': 2, 'bo5': 1}
        
        # Merge features back
        df = df.merge(match_round_stats[['match_id', 'estimated_rounds_per_map', 'game_length_category']], 
                     on='match_id', how='left')
        df = df.merge(match_variance[['match_id', 'kill_cv', 'matchup_evenness']], 
                     on='match_id', how='left')
        
        df['series_pressure'] = df['series_type'].map(series_pressure).fillna(2)
        
        print(f"   ✅ Added matchup features")
        return df
    
    def calculate_player_role_features(self, df):
        """Identify player roles and calculate role-specific features"""
        print("🎭 CALCULATING PLAYER ROLES...")
        
        # Calculate player performance percentiles
        player_stats = df.groupby('consolidated_player_name').agg({
            'kills': ['mean', 'std', 'count'],
            'acs': 'mean',
            'fk': 'mean',
            'kdr': 'mean'
        }).reset_index()
        
        player_stats.columns = ['player_name', 'avg_kills', 'kills_std', 'total_maps', 'avg_acs', 'avg_fk', 'avg_kdr']
        
        # Only consider players with enough data
        experienced_players = player_stats[player_stats['total_maps'] >= 10].copy()
        
        if len(experienced_players) > 0:
            # Calculate percentiles for classification
            experienced_players['kills_percentile'] = experienced_players['avg_kills'].rank(pct=True)
            experienced_players['acs_percentile'] = experienced_players['avg_acs'].rank(pct=True)
            experienced_players['fk_percentile'] = experienced_players['avg_fk'].rank(pct=True)
            
            # Classify player roles
            def classify_player_role(row):
                kills_pct = row['kills_percentile']
                acs_pct = row['acs_percentile']
                fk_pct = row['fk_percentile']
                
                # Star fragger: Top 20% in kills AND (top 30% in ACS OR FK)
                if kills_pct >= 0.8 and (acs_pct >= 0.7 or fk_pct >= 0.7):
                    return 'star_fragger'
                elif kills_pct >= 0.6:
                    return 'secondary_fragger'  
                elif kills_pct <= 0.4:
                    return 'support_player'
                else:
                    return 'balanced_player'
            
            experienced_players['player_role'] = experienced_players.apply(classify_player_role, axis=1)
            
            # Consistency analysis
            experienced_players['consistency_score'] = 1 / (1 + experienced_players['kills_std'])
            experienced_players['consistency_tier'] = pd.qcut(experienced_players['consistency_score'], 
                                                            q=3, labels=['volatile', 'moderate', 'consistent'])
            
            # Create mappings
            role_mapping = dict(zip(experienced_players['player_name'], experienced_players['player_role']))
            consistency_mapping = dict(zip(experienced_players['player_name'], experienced_players['consistency_tier']))
            
        else:
            role_mapping = {}
            consistency_mapping = {}
        
        # Apply to main dataframe
        df['player_role'] = df['consolidated_player_name'].map(role_mapping).fillna('unknown')
        df['consistency_tier'] = df['consolidated_player_name'].map(consistency_mapping).fillna('moderate')
        
        # Role-specific expectations
        role_expectations = {
            'star_fragger': 1.3,
            'secondary_fragger': 1.1,
            'balanced_player': 1.0,
            'support_player': 0.8,
            'unknown': 1.0
        }
        
        df['role_kill_multiplier'] = df['player_role'].map(role_expectations)
        df['facing_star_fraggers'] = 0.5  # Placeholder
        
        print(f"   ✅ Identified roles: {df['player_role'].value_counts().to_dict()}")
        return df
    
    def calculate_meta_features(self, df):
        """Calculate advanced meta-game features"""
        print("🧠 CALCULATING META-GAME FEATURES...")
        
        # Team synergy
        team_performance = df.groupby(['team_name', 'match_id'])['kills'].sum().reset_index()
        team_avg_kills = team_performance.groupby('team_name')['kills'].mean().to_dict()
        df['team_avg_kills'] = df['team_name'].map(team_avg_kills).fillna(df['kills'].mean())
        
        # Map specialization
        player_map_performance = df.groupby(['consolidated_player_name', 'map_name'])['kills'].mean().reset_index()
        overall_player_performance = df.groupby('consolidated_player_name')['kills'].mean().reset_index()
        overall_player_performance.columns = ['consolidated_player_name', 'overall_avg_kills']
        
        map_specialization = player_map_performance.merge(overall_player_performance, on='consolidated_player_name')
        map_specialization['map_specialization_factor'] = map_specialization['kills'] / map_specialization['overall_avg_kills']
        
        map_spec_lookup = {}
        for _, row in map_specialization.iterrows():
            key = (row['consolidated_player_name'], row['map_name'])
            map_spec_lookup[key] = row['map_specialization_factor']
        
        df['map_specialization'] = df.apply(
            lambda row: map_spec_lookup.get((row['consolidated_player_name'], row['map_name']), 1.0), 
            axis=1
        )
        
        # Form trend (simplified)
        df['form_trend'] = 0  # Placeholder for now
        
        print("   ✅ Added meta features")
        return df
    
    def calculate_head_to_head_features(self, df):
        """Calculate head-to-head matchup features"""
        print("🥊 CALCULATING HEAD-TO-HEAD MATCHUPS...")
        
        # Sort by date to ensure proper chronological order
        df = df.sort_values(['match_date', 'match_id', 'map_id']).reset_index(drop=True)
        
        # Get opponent teams for each match
        match_teams = df.groupby('match_id')['team_name'].unique().reset_index()
        opponent_mapping = {}
        
        for _, row in match_teams.iterrows():
            match_id = row['match_id']
            teams = row['team_name']
            if len(teams) >= 2:
                team1, team2 = teams[0], teams[1]
                opponent_mapping[f"{match_id}_{team1}"] = team2
                opponent_mapping[f"{match_id}_{team2}"] = team1
        
        # Add opponent team to each record
        df['opponent_team'] = df.apply(
            lambda row: opponent_mapping.get(f"{row['match_id']}_{row['team_name']}", 'unknown'),
            axis=1
        )
        
        # Calculate head-to-head performance (avoiding data leakage)
        h2h_features = []
        
        for i, row in df.iterrows():
            if i % 50000 == 0:
                print(f"   Processing head-to-head for record {i:,}/{len(df):,}")
                
            player = row['consolidated_player_name']
            opponent_team = row['opponent_team']
            current_match_date = row['match_date']
            current_map = row['map_name']
            
            # Get historical performance against this opponent team (before current match)
            historical_vs_opponent = df[
                (df['consolidated_player_name'] == player) &
                (df['opponent_team'] == opponent_team) &
                (df['match_date'] < current_match_date)
            ]
            
            if len(historical_vs_opponent) >= 3:
                h2h_avg_kills = historical_vs_opponent['kills'].mean()
                h2h_kill_trend = historical_vs_opponent.tail(5)['kills'].mean() - historical_vs_opponent.head(5)['kills'].mean()
                h2h_map_performance = historical_vs_opponent[
                    historical_vs_opponent['map_name'] == current_map
                ]['kills'].mean() if len(historical_vs_opponent[historical_vs_opponent['map_name'] == current_map]) > 0 else h2h_avg_kills
            elif len(historical_vs_opponent) >= 1:
                h2h_avg_kills = historical_vs_opponent['kills'].mean()
                h2h_kill_trend = 0
                h2h_map_performance = h2h_avg_kills
            else:
                h2h_avg_kills = row['hist_avg_kills']  # Fall back to overall average
                h2h_kill_trend = 0
                h2h_map_performance = row['hist_avg_kills']
            
            h2h_features.append({
                'h2h_avg_kills': h2h_avg_kills,
                'h2h_kill_trend': h2h_kill_trend,
                'h2h_map_performance': h2h_map_performance,
                'h2h_matches_played': len(historical_vs_opponent)
            })
        
        # Add features to dataframe
        h2h_df = pd.DataFrame(h2h_features)
        for col in h2h_df.columns:
            df[col] = h2h_df[col]
        
        print("   ✅ Added head-to-head features")
        return df
    
    def calculate_momentum_features(self, df):
        """Calculate recent form and momentum features"""
        print("📈 CALCULATING RECENT FORM MOMENTUM...")
        
        # Sort by player and date
        df = df.sort_values(['consolidated_player_name', 'match_date', 'map_id']).reset_index(drop=True)
        
        # Calculate momentum features using vectorized operations
        momentum_features = []
        
        # Group by player for efficient calculation
        for player, player_df in df.groupby('consolidated_player_name'):
            player_df = player_df.reset_index(drop=True)
            
            # Calculate rolling statistics (with shift to avoid data leakage)
            recent_3_kills = player_df['kills'].rolling(3, min_periods=1).mean().shift(1).fillna(player_df['kills'].iloc[0] if len(player_df) > 0 else 15.0)
            recent_5_kills = player_df['kills'].rolling(5, min_periods=1).mean().shift(1).fillna(player_df['kills'].iloc[0] if len(player_df) > 0 else 15.0)
            
            # Momentum trend (recent vs earlier performance)
            momentum_trend = recent_3_kills - recent_5_kills
            
            # Kill consistency (rolling standard deviation)
            kill_consistency = 1 / (1 + player_df['kills'].rolling(5, min_periods=1).std().shift(1).fillna(1.0))
            
            # Hot/cold streak detection
            hot_streak = (recent_3_kills > player_df['kills'].rolling(10, min_periods=1).mean().shift(1)).astype(int)
            
            # Performance vs expectation
            expected_kills = player_df['kills'].expanding().mean().shift(1).fillna(15.0)
            recent_vs_expected = recent_3_kills - expected_kills
            
            # Store features for this player
            for i in range(len(player_df)):
                momentum_features.append({
                    'recent_momentum_3': recent_3_kills.iloc[i],
                    'momentum_trend': momentum_trend.iloc[i],
                    'kill_consistency': kill_consistency.iloc[i],
                    'hot_streak': hot_streak.iloc[i],
                    'recent_vs_expected': recent_vs_expected.iloc[i]
                })
        
        # Add features to dataframe
        momentum_df = pd.DataFrame(momentum_features)
        for col in momentum_df.columns:
            df[col] = momentum_df[col]
        
        print("   ✅ Added momentum features")
        return df
    
    def calculate_map_opponent_features(self, df):
        """Calculate map-specific opponent analysis"""
        print("🗺️ CALCULATING MAP-SPECIFIC OPPONENT ANALYSIS...")
        
        map_opponent_features = []
        
        for i, row in df.iterrows():
            if i % 50000 == 0:
                print(f"   Processing map-opponent analysis for record {i:,}/{len(df):,}")
                
            player = row['consolidated_player_name']
            opponent_team = row['opponent_team']
            current_map = row['map_name']
            current_match_date = row['match_date']
            
            # Historical performance on this specific map against this opponent
            map_opponent_history = df[
                (df['consolidated_player_name'] == player) &
                (df['opponent_team'] == opponent_team) &
                (df['map_name'] == current_map) &
                (df['match_date'] < current_match_date)
            ]
            
            # Map performance against all opponents (for comparison)
            map_overall_history = df[
                (df['consolidated_player_name'] == player) &
                (df['map_name'] == current_map) &
                (df['match_date'] < current_match_date)
            ]
            
            # Opponent difficulty on this map (how they generally perform)
            opponent_map_strength = df[
                (df['team_name'] == opponent_team) &
                (df['map_name'] == current_map) &
                (df['match_date'] < current_match_date)
            ]['kills'].mean() if len(df[
                (df['team_name'] == opponent_team) &
                (df['map_name'] == current_map) &
                (df['match_date'] < current_match_date)
            ]) > 0 else df['kills'].mean()
            
            if len(map_opponent_history) >= 2:
                map_opponent_avg = map_opponent_history['kills'].mean()
                map_opponent_consistency = 1 / (1 + map_opponent_history['kills'].std())
            elif len(map_overall_history) >= 3:
                map_opponent_avg = map_overall_history['kills'].mean()
                map_opponent_consistency = 1 / (1 + map_overall_history['kills'].std())
            else:
                map_opponent_avg = row['hist_avg_kills']
                map_opponent_consistency = 0.5
            
            # Map difficulty factor (how challenging this map generally is)
            overall_map_avg = df[df['map_name'] == current_map]['kills'].mean()
            map_difficulty = overall_map_avg / df['kills'].mean() if df['kills'].mean() > 0 else 1.0
            
            map_opponent_features.append({
                'map_opponent_avg': map_opponent_avg,
                'map_opponent_consistency': map_opponent_consistency,
                'opponent_map_strength': opponent_map_strength,
                'map_difficulty_factor': map_difficulty,
                'map_opponent_experience': len(map_opponent_history)
            })
        
        # Add features to dataframe
        map_opp_df = pd.DataFrame(map_opponent_features)
        for col in map_opp_df.columns:
            df[col] = map_opp_df[col]
        
        print("   ✅ Added map-opponent features")
        return df

class AdvancedTrainer:
    """Advanced trainer with sophisticated feature engineering"""
    
    def __init__(self, db_path: str):
        self.db_path = db_path
        self.scaler = StandardScaler()
        self.consolidator = PlayerConsolidator()
        self.feature_engineer = AdvancedFeatureEngineer()
        
    def train_advanced_model(self):
        """Train model with all advanced features"""
        print("🎯 STARTING ADVANCED TRAINING WITH ALL SOPHISTICATED FEATURES")
        print("=" * 80)
        
        # Load ALL data (bypass date filter)
        loader = FastDatabaseDataLoader(db_path=self.db_path)
        
        query = """
        SELECT
            p.name as player_name, t.name as team_name, pms.team_id as team_id,
            m.match_date, m.series_type, tour.name as tournament_name,
            mp.map_name, pms.kills, pms.deaths, pms.assists, pms.acs, pms.adr,
            pms.fk, pms.hs_percentage, pms.kdr, m.match_id, pms.map_id
        FROM player_match_stats pms
        JOIN players p ON pms.player_id = p.id
        JOIN teams t ON pms.team_id = t.id
        JOIN matches m ON pms.match_id = m.id
        JOIN maps mp ON pms.map_id = mp.id
        JOIN tournaments tour ON m.tournament_id = tour.id
        ORDER BY p.name, m.match_date, pms.map_id
        """
        
        with loader.get_connection() as conn:
            df = pd.read_sql_query(query, conn)
        
        print(f"📊 Loaded {len(df)} records from {df['player_name'].nunique()} players")
        
        # Player consolidation and confidence weighting
        df = self.consolidator.consolidate_players(df)
        
        # Calculate confidence weights
        player_map_counts = df['consolidated_player_name'].value_counts()
        weights = {}
        for player, count in player_map_counts.items():
            if count >= 20:
                weight = 1.0
            elif count >= 10:
                weight = 0.8
            elif count >= 5:
                weight = 0.6
            elif count >= 3:
                weight = 0.4
            else:
                weight = 0.2
            weights[player] = weight
        
        df['player_weight'] = df['consolidated_player_name'].map(weights)
        
        # ⭐ ADVANCED FEATURE ENGINEERING
        df = self.feature_engineer.calculate_matchup_features(df)
        df = self.feature_engineer.calculate_player_role_features(df) 
        df = self.feature_engineer.calculate_meta_features(df)
        
        # 🚀 NEW ELITE FEATURES
        df = self.feature_engineer.calculate_head_to_head_features(df)
        df = self.feature_engineer.calculate_momentum_features(df)
        df = self.feature_engineer.calculate_map_opponent_features(df)
        
        # Basic feature engineering
        print("\\n⚡ CALCULATING BASIC FEATURES...")
        df = loader.calculate_map_features_FAST(df)
        
        # Prepare advanced feature set
        advanced_features = [
            # Basic features
            'hist_avg_kills', 'hist_avg_kdr', 'recent_kills_5', 'days_since_last', 'series_importance',
            # Matchup dynamics  
            'estimated_rounds_per_map', 'kill_cv', 'series_pressure',
            # Player role features
            'role_kill_multiplier', 'facing_star_fraggers',
            # Meta features
            'team_avg_kills', 'map_specialization', 'form_trend',
            # 🥊 Head-to-head features
            'h2h_avg_kills', 'h2h_kill_trend', 'h2h_map_performance', 'h2h_matches_played',
            # 📈 Momentum features  
            'recent_momentum_3', 'momentum_trend', 'kill_consistency', 'hot_streak', 'recent_vs_expected',
            # 🗺️ Map-opponent features
            'map_opponent_avg', 'map_opponent_consistency', 'opponent_map_strength', 'map_difficulty_factor', 'map_opponent_experience'
        ]
        
        # Encode categorical features
        le_game_length = LabelEncoder()
        df['game_length_encoded'] = le_game_length.fit_transform(df['game_length_category'].fillna('medium_game'))
        advanced_features.append('game_length_encoded')
        
        le_evenness = LabelEncoder() 
        df['matchup_evenness_encoded'] = le_evenness.fit_transform(df['matchup_evenness'].fillna('somewhat_even'))
        advanced_features.append('matchup_evenness_encoded')
        
        le_role = LabelEncoder()
        df['player_role_encoded'] = le_role.fit_transform(df['player_role'].fillna('balanced_player'))
        advanced_features.append('player_role_encoded')
        
        le_consistency = LabelEncoder()
        df['consistency_encoded'] = le_consistency.fit_transform(df['consistency_tier'].fillna('moderate'))
        advanced_features.append('consistency_encoded')
        
        # Filter available features
        available_features = [col for col in advanced_features if col in df.columns]
        print(f"\\n🎯 Using {len(available_features)} advanced features")
        
        # Prepare training data
        X = df[available_features].fillna(0).values
        y = df['kills'].values
        weights_array = df['player_weight'].values
        
        print(f"\\n✅ Advanced training data prepared:")
        print(f"   Samples: {len(X):,}")
        print(f"   Features: {len(available_features)}")
        print(f"   Consolidated players: {df['consolidated_player_name'].nunique():,}")
        
        # Training
        X_train, X_test, y_train, y_test, w_train, w_test = train_test_split(
            X, y, weights_array, test_size=0.2, random_state=42
        )
        
        X_train_scaled = self.scaler.fit_transform(X_train)
        X_test_scaled = self.scaler.transform(X_test)
        
        # Convert to tensors
        X_train_tensor = torch.FloatTensor(X_train_scaled).to(device)
        y_train_tensor = torch.FloatTensor(y_train).to(device)
        w_train_tensor = torch.FloatTensor(w_train).to(device)
        X_test_tensor = torch.FloatTensor(X_test_scaled).to(device)
        y_test_tensor = torch.FloatTensor(y_test).to(device)
        
        # Advanced model architecture
        input_size = X_train_tensor.shape[1]
        model = KillPredictionNN(input_size, hidden_sizes=[256, 128, 64, 32]).to(device)
        
        # Weighted loss function
        def weighted_mse_loss(predictions, targets, weights):
            squared_errors = (predictions - targets) ** 2
            weighted_errors = squared_errors * weights
            return torch.mean(weighted_errors)
        
        optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
        
        # Training setup
        class WeightedDataset:
            def __init__(self, X, y, weights):
                self.X = X
                self.y = y
                self.weights = weights
            
            def __len__(self):
                return len(self.X)
            
            def __getitem__(self, idx):
                return self.X[idx], self.y[idx], self.weights[idx]
        
        train_dataset = WeightedDataset(X_train_tensor.cpu().numpy(), 
                                      y_train_tensor.cpu().numpy(), 
                                      w_train_tensor.cpu().numpy())
        train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
        
        # Training loop
        print(f"\\n🧠 TRAINING ADVANCED MODEL...")
        best_mae = float('inf')
        patience = 0
        
        for epoch in range(100):
            model.train()
            train_loss = 0.0
            
            for batch_X, batch_y, batch_w in train_loader:
                batch_X = batch_X.to(device)
                batch_y = batch_y.to(device) 
                batch_w = batch_w.to(device)
                
                optimizer.zero_grad()
                outputs = model(batch_X).squeeze()
                loss = weighted_mse_loss(outputs, batch_y, batch_w)
                loss.backward()
                optimizer.step()
                train_loss += loss.item()
            
            if epoch % 10 == 0:
                model.eval()
                with torch.no_grad():
                    val_outputs = model(X_test_tensor).squeeze()
                    val_mae = mean_absolute_error(y_test_tensor.cpu().numpy(), 
                                                val_outputs.cpu().numpy())
                
                print(f"Epoch {epoch}: Weighted Loss = {train_loss/len(train_loader):.4f}, MAE = {val_mae:.3f}")
                
                # Early stopping
                if val_mae < best_mae:
                    best_mae = val_mae
                    patience = 0
                else:
                    patience += 1
                    if patience >= 3:
                        print(f"Early stopping at epoch {epoch}")
                        break
        
        # Final evaluation
        model.eval()
        with torch.no_grad():
            y_pred = model(X_test_tensor).squeeze().cpu().numpy()
            y_test_np = y_test_tensor.cpu().numpy()
        
        mse = mean_squared_error(y_test_np, y_pred)
        mae = mean_absolute_error(y_test_np, y_pred)
        r2 = r2_score(y_test_np, y_pred)
        
        # Results analysis
        print(f"\\n🎉 ADVANCED TRAINING COMPLETED!")
        print(f"🎯 MAE: {mae:.3f} kills per map")
        print(f"📈 R²: {r2:.6f}")
        
        # Achievement analysis
        if mae <= 3.0:
            print(f"\\n🏆 OUTSTANDING! MAE of {mae:.2f} is approaching your 1-3 target!")
        elif mae <= 5.0:
            print(f"\\n🎉 EXCELLENT! MAE of {mae:.2f} is substantial improvement!")
        elif mae <= 6.5:
            print(f"\\n🎯 GOOD PROGRESS! MAE of {mae:.2f} shows advanced features help!")
        
        # Save advanced model
        os.makedirs('models', exist_ok=True)
        model_data = {
            'model_state_dict': model.state_dict(),
            'input_size': input_size,
            'hidden_sizes': [256, 128, 64, 32],
            'scaler': self.scaler,
            'feature_columns': available_features,
            'performance': {'mse': mse, 'mae': mae, 'r2': r2}
        }
        
        joblib.dump(model_data, 'models/enhanced_elite_model.pkl')
        print(f"\\n✅ Enhanced model saved as 'enhanced_elite_model.pkl'!")
        
        return model_data

# Usage
if 'uploaded' in globals() and uploaded:
    db_path = list(uploaded.keys())[0]
    print(f"🎯 Starting ADVANCED training with: {db_path}")
    
    try:
        trainer = AdvancedTrainer(db_path=db_path)
        model_data = trainer.train_advanced_model()
        
        print("\\n🎉 SUCCESS! Your model now features:")
        print("   ✅ Matchup dynamics (even vs uneven games)")
        print("   ✅ Player role modeling (star fraggers vs supports - no bias)")
        print("   ✅ Game length prediction (more rounds = more kills)")
        print("   ✅ Team synergy effects")
        print("   ✅ Map specialization")
        print("   ✅ 🥊 Head-to-head opponent analysis")
        print("   ✅ 📈 Recent form momentum & hot streaks")
        print("   ✅ 🗺️ Map-specific opponent performance")
        print("   ✅ Professional-grade feature engineering!")
        
    except Exception as e:
        print(f"❌ Training failed: {e}")
        import traceback
        traceback.print_exc()
else:
    print("❌ Please upload your database file first (run Cell 3)")

print("\\n💡 ADVANCED FEATURES EXPLAINED:")
print("   🎮 Game Length: Long games = more rounds = higher kill totals")
print("   ⚖️ Matchup Evenness: Close games vs stomps affect individual performance")
print("   🎭 Player Roles: Star fraggers vs support players (no bias - purely stats-based)")
print("   🗺️ Map Specialization: Player performance varies by map")
print("   🤝 Team Synergy: How well teams work together affects individual stats")
print("   🏆 Series Pressure: BO1 vs BO3 vs BO5 psychological effects")
print("   🥊 Head-to-Head: How players perform specifically against certain opponents")
print("   📈 Momentum: Recent form trends, hot/cold streaks, consistency patterns")
print("   🗺️🥊 Map-Opponent: Performance on specific maps against specific teams")


In [ ]:
# 📥 Cell 6: Download Trained Model

try:
    from google.colab import files
    
    if os.path.exists('models/enhanced_elite_model.pkl'):
        print("📦 Downloading your enhanced elite trained model...")
        files.download('models/enhanced_elite_model.pkl')
        print("✅ Model downloaded successfully!")
        print("\\n🎯 You can now use this model to make kill predictions!")
        print("\\n📋 What you got:")
        print("  🧠 Enhanced neural network model with elite features")
        print("  📊 Feature scaler and encoders") 
        print("  📈 Performance metrics")
        print("  🎭 Player role modeling (unbiased)")
        print("  ⚖️ Matchup dynamics analysis")
        print("  🗺️ Map specialization factors")
        print("  🥊 Head-to-head opponent analysis")
        print("  📈 Recent form momentum tracking")
        print("  🗺️🥊 Map-specific opponent performance")
    else:
        print("❌ No trained model found. Please run Cell 5 first.")
        
except Exception as e:
    print(f"❌ Download error: {e}")
    print("💡 You can find the model in the files panel on the left.")
